In [1]:
import pandas as pd
import numpy as np
import requests                    
import json
from sklearn.cluster import KMeans
import plotly.express as px 
from tkinter import *
from PIL import Image
from PIL import Image, ImageTk
import os


In [2]:
##### INTERFACE #####
# Nesse exemplo, os dados são salvos em imagens e depois reproduzidos na interface.

In [2]:
df = pd.read_csv('data/final.csv')   #dados 2010

def mapa(ind,valor,cl1,uf):
    
    #df['ind'].astype(int)
    #df['valor'].astype(int)
    
    df_mask=df['ind'] > ind
    df_new = df[df_mask]
    
    df_mask=df_new['valor']>valor
    df_new = df_new[df_mask]
    
    if uf != 0 :
        df_mask=df_new['UF']== uf
        df_new = df_new[df_mask]
    
    X = df_new[['latitude', 'longitude']].to_numpy()
    
    
    cl = KMeans(n_clusters=cl1, random_state=0).fit(X) # clusters manuais ()
    clusters = pd.DataFrame(cl.labels_, columns=['cluster'])
    clusters2 = pd.DataFrame(cl.cluster_centers_, columns=['lat','lng'])
    a = len(clusters)
    

    i=0
    j = 0
    clusters3 = clusters.copy()
    lista = []
    lista2 = []
    for j in range(a):
        for i in range(cl1):
            if clusters.cluster.iloc[j] == i:
                lista.append(clusters2.lat.iloc[i])  
                lista2.append(clusters2.lng.iloc[i]) 
            i = i+1
        j = j+1
        
    clusters3['lat'] = lista
    clusters3['lng'] = lista2 

    global detail
    detail = pd.merge(left=df_new.reset_index(), right=clusters3, left_index=True, right_index=True)
    detail = detail.astype({'cluster': 'str'})

def scart_centroid ():
    fig = px.scatter(detail, x="lat", y="lng", color="cluster",hover_name="CIDADE",title='Centroid') # onde colocar a filial
    fig.write_image("fig1.png")
    img = Image.open('fig1.png')
    
    janela =Toplevel ()
    img=ImageTk.PhotoImage(img)
    label= Label(janela, image= img)
    label.image= img
    label.pack()
    janela.mainloop()

    
def scart ():
    fig = px.scatter(detail, x="latitude", y="longitude", color="cluster",hover_name="CIDADE",title='IdososxPoup')
    fig.write_image("fig2.png")    
    img = Image.open('fig2.png')
    
    janela =Toplevel ()
    img=ImageTk.PhotoImage(img)
    label= Label(janela, image= img)
    label.image= img
    label.pack()
    janela.mainloop()

def mapa_color():
    if uf==0:
        fig = px.scatter_mapbox(detail, lat="latitude", lon="longitude", hover_name="CIDADE", hover_data=["valor","ind"],
                                color='valor', zoom=2, height=300)
    else:
        fig = px.scatter_mapbox(detail, lat="latitude", lon="longitude", hover_name="CIDADE", hover_data=["valor","ind"],
                                color='valor', zoom=6, height=300)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.write_image("fig3.png")    
    img = Image.open('fig3.png')
    
    janela=Toplevel ()
    img=ImageTk.PhotoImage(img)
    label= Label(janela, image= img)
    label.image= img
    label.pack()
    janela.mainloop()


def mapa_color2():
    if uf==0:
        fig = px.scatter_mapbox(detail, lat="lat", lon="lng", hover_name="CIDADE", hover_data=["valor","ind"],
                                color='valor',range_color=(0, 30000000), zoom=2, height=300)
    else:
        fig = px.scatter_mapbox(detail, lat="lat", lon="lng", hover_name="CIDADE", hover_data=["valor","ind"],
                                color='valor',range_color=(0, 30000000), zoom=6, height=300)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.write_image("fig4.png")    
    img = Image.open('fig4.png')
    
    janela=Toplevel ()
    img=ImageTk.PhotoImage(img)
    label= Label(janela, image= img)
    label.image= img
    label.pack()
    janela.mainloop()

def bt_onclick ():
    if(str(ind.get()).isnumeric() and str (cluster.get()).isnumeric() and str(valor.get())).isnumeric():
        global uf
        uf = str(estado.get())
        if(len(uf) != 0):
            uf = uf.upper()
        else:
            uf = 0
        indice = int(ind.get())
        valor_poup = int(valor.get())
        cl1 = int(cluster.get())
    else:
        lb["text"] = 'invalido'
        mapa(indice,valor_poup,cl1,uf)
    
    janela = Tk()      

        
    bt1= Button(janela,text="Scarter", command=scart)
    bt1.place(x=40,y=10)

    bt2= Button(janela,text="Scarter centroids",command=scart_centroid)
    bt2.place(x=40,y=40)

    bt3= Button(janela,text="Mapa",command=mapa_color)
    bt3.place(x=40,y=70)
    
    bt4= Button(janela,text="Mapa2",command=mapa_color2)
    bt4.place(x=40,y=100)

    janela.geometry("150x150")
    janela.title("Escolha os plots")
    janela.mainloop()

# Interface Grafica :

root = Tk()

ind = Entry(root)
ind.place(x=80,y=40)
label_ind = Label(root,text="% idosos: ")
label_ind.place(x=10,y=40)


valor = Entry(root)
valor.place(x=80,y=80)
label_valor = Label(root,text="Poupança: ")
label_valor.place(x=10,y=80)

cluster = Entry(root)
cluster.place(x=80,y=120)
label_cluster = Label(root,text="Clusters: ")
label_cluster.place(x=10,y=120)

estado = Entry(root)
estado.place(x=80,y=160)
label_estado = Label(root,text="Sigla, XX: ")
label_estado.place(x=10,y=160)


bt= Button(root,text="enviar",command=bt_onclick)
bt.place(x=80,y=200)


root.geometry("250x250")
root.title("Gerar mapa")

root.mainloop()
